In [1]:
import pandas
from google import genai
from google.genai import types # type definitions and configurations

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# retrieve the api keys
serp_api_key = os.getenv("SERP_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

In [7]:
# initialize gemini client
google_client = genai.Client(api_key=google_api_key)

In [8]:
response = google_client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Provide a high level overview of model context protocol under 150 words.")

print(response.text)

Model context protocols provide a standardized way for different machine learning models and tools to interact and share information, improving collaboration and reusability. Imagine them as agreements defining how models "talk" to each other.

These protocols standardize data formats, model metadata (like input/output types), and execution methods. This allows tools to easily discover, understand, and utilize different models without needing custom code for each. By defining a common language for models, context protocols enable features like model chaining, automated model selection, and model explainability, fostering a more modular and efficient ML ecosystem. They essentially make models plug-and-play components.



In [3]:
# load mcp libraries
from mcp import ClientSession, StdioServerParameters 
from mcp.client.stdio import stdio_client # to establish connection with mcp server over standard I/O

In [5]:
# configure mcp server tool
server_params = StdioServerParameters(
    command="mcp-flight-search", # points to mcp-flight-search server module 
    args=["--connection_type", "stdio"],
    env={"SERP_API_KEY": os.getenv("SERP_API_KEY")}
)

In [11]:
# MCP server connection and listing tools

async def run():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            prompt = f"Find flights from Raleigh to Miami 2025-06-19"
            
            # triggers initial mcp handshake between client and server
            await session.initialize() 

            # requests list of all tools
            mcp_tools = await session.list_tools()

            tools = [
                types.Tool(
                    function_declarations=[
                        {
                            "name": tool.name,
                            "description": tool.description,
                            "parameters": {
                                k: v
                                for k,v in tool.inputSchema.items()
                                if k not in ["additionalProperties", "$schema"]
                            },
                        }
                    ]
                )
                for tool in mcp_tools.tools
            ]

            # connect prompt and tools to gemini flash api
            response = google_client.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt,
                config=types.GenerateContentConfig(
                    temperature=0,
                    tools=tools
                ),
            )
    #return response
        